In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import LSTM

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


In [2]:
df = pd.read_csv('D:\MY WORK\Turing Internship\DATA\Google_train_data.csv')
df.head()

<>:1: SyntaxWarning: invalid escape sequence '\M'
<>:1: SyntaxWarning: invalid escape sequence '\M'
D:\Temp\ipykernel_14492\1447379173.py:1: SyntaxWarning: invalid escape sequence '\M'
  df = pd.read_csv('D:\MY WORK\Turing Internship\DATA\Google_train_data.csv')


,Date,Open,High,Low,Close,Volume
0,1/3/2012,325.25,332.83,324.97,663.59,"7,380,500"
1,1/4/2012,331.27,333.87,329.08,666.45,"5,749,400"
2,1/5/2012,329.83,330.75,326.89,657.21,"6,590,300"
3,1/6/2012,328.34,328.77,323.68,648.24,"5,405,900"
4,1/9/2012,322.04,322.29,309.46,620.76,"11,688,800"


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1258 entries, 0 to 1257
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    1258 non-null   object 
 1   Open    1258 non-null   float64
 2   High    1258 non-null   float64
 3   Low     1258 non-null   float64
 4   Close   1258 non-null   object 
 5   Volume  1258 non-null   object 
dtypes: float64(3), object(3)
memory usage: 59.1+ KB


In [4]:
df.drop(['Date'],axis='columns',inplace=True)

In [5]:
df.dtypes

Open      float64
High      float64
Low       float64
Close      object
Volume     object
dtype: object

In [6]:
df['Volume'] = df['Volume'].str.replace(',','')
df['Volume'] = df['Volume'].astype('float32')
df['Close'] = df['Close'].str.replace(',','')
df['Close'] = df['Close'].astype('float32')
df['Open'] = df['Open'].astype('float32')
df['High'] = df['High'].astype('float32')
df['Low'] = df['Low'].astype('float32')

In [7]:
df.dtypes

Open      float32
High      float32
Low       float32
Close     float32
Volume    float32
dtype: object

In [8]:
df.shape

(1258, 5)

In [9]:
df.isnull().sum()

Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

In [10]:
df.isna().sum()

Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

In [11]:
correlation = df.corr()
print(correlation['Close'].sort_values(ascending=False))

Close     1.000000
Volume    0.192357
Low       0.129001
Open      0.125832
High      0.124351
Name: Close, dtype: float64


In [12]:
X = df[['Open', 'High', 'Low', 'Volume']]

y = df['Close']

In [13]:
#Running LSTM
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [14]:
X_reshape = X_scaled.reshape((X_scaled.shape[0],1,X_scaled.shape[1]))

In [15]:
from sklearn.model_selection import KFold
fold = KFold(n_splits=10)

for train_index, test_index in fold.split(X_reshape, y):
    X_train, X_test = X_reshape[train_index], X_reshape[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [16]:
X_train.shape,y_train.shape

((1133, 1, 4), (1133,))

In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(100,activation='relu',input_shape =(1, 4)),
    tf.keras.layers.Dense(80,activation='relu'),
    tf.keras.layers.Dense(60,activation='relu'),
    tf.keras.layers.Dense(40,activation='relu'),
    tf.keras.layers.Dense(20,activation='relu'),
    tf.keras.layers.Dense(1)
])

C:\Users\Archit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [18]:
model.compile(optimizer='adam',loss='mae')

In [19]:
model.fit(X_train, y_train, epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 717.1807 - val_loss: 768.3580
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 707.7051 - val_loss: 696.5844
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 588.7248 - val_loss: 345.8206
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 252.9942 - val_loss: 95.9083
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 197.2237 - val_loss: 11.5501
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 163.4980 - val_loss: 16.1740
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 144.1378 - val_loss: 27.5888
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 121.6899 - val_loss: 57.3840
Epoch 9/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 118.9404 - val_loss: 59.5148
Epoch 10/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 110.8590 - val_loss: 57.5555
Epoch 11/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 112.1033 - val_loss: 38.7645
Epoch 12/100
36/36 ━━━━━━━

In [20]:
model.evaluate(X_test,y_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.8391 


5.517039775848389

In [21]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_pred

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [22]:
from sklearn.metrics import mean_absolute_error

from sklearn.metrics import mean_squared_error
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

Mean Absolute Error: 770.3359189453125


In [23]:
df1 = pd.read_csv('D:\MY WORK\Turing Internship\DATA\Google_test_data.csv')
df1

<>:1: SyntaxWarning: invalid escape sequence '\M'
<>:1: SyntaxWarning: invalid escape sequence '\M'
D:\Temp\ipykernel_14492\2989969055.py:1: SyntaxWarning: invalid escape sequence '\M'
  df1 = pd.read_csv('D:\MY WORK\Turing Internship\DATA\Google_test_data.csv')


,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-08-13,1236.979980,1249.272949,1233.640991,1235.010010,1235.010010,997300
1,2018-08-14,1235.189941,1245.869995,1225.109985,1242.099976,1242.099976,1348100
2,2018-08-15,1229.260010,1235.239990,1209.510010,1214.380005,1214.380005,1828800
3,2018-08-16,1224.729980,1226.000000,1202.550049,1206.489990,1206.489990,1343200
4,2018-08-17,1202.030029,1209.020020,1188.239990,1200.959961,1200.959961,1389600
...,...,...,...,...,...,...,...
247,2019-08-07,1156.000000,1178.444946,1149.624023,1173.989990,1173.989990,1444300
248,2019-08-08,1182.829956,1205.010010,1173.020020,1204.800049,1204.800049,1468000
249,2019-08-09,1197.989990,1203.880005,1183.603027,1188.010010,1188.010010,1065700
250,2019-08-12,1179.209961,1184.959961,1167.671997,1174.709961,1174.709961,1003000


In [24]:
df1.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [25]:
df1.drop(['Adj Close','Date'],axis='columns',inplace=True)

In [26]:
df1.head()

,Open,High,Low,Close,Volume
0,1236.979980,1249.272949,1233.640991,1235.010010,997300
1,1235.189941,1245.869995,1225.109985,1242.099976,1348100
2,1229.260010,1235.239990,1209.510010,1214.380005,1828800
3,1224.729980,1226.000000,1202.550049,1206.489990,1343200
4,1202.030029,1209.020020,1188.239990,1200.959961,1389600


In [27]:
df1.dtypes

Open      float64
High      float64
Low       float64
Close     float64
Volume      int64
dtype: object

In [28]:
df1.isnull().sum()

Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

In [29]:
df1.isna().sum()

Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

In [30]:
X_new = df1[['Open', 'High', 'Low', 'Volume']]

y_new = df1['Close']

In [31]:
scaler = MinMaxScaler()
X_scaled_new = scaler.fit_transform(X_new)

In [32]:
X_reshape_new = X_scaled_new.reshape((X_scaled_new.shape[0],1,X_scaled_new.shape[1]))

In [33]:
foldN = KFold(n_splits=10)

for train_index_new, test_index_new in foldN.split(X_reshape_new, y_new):
    X_new_train, X_new_test = X_reshape_new[train_index_new], X_reshape_new[test_index_new]
    y_new_train, y_new_test = y_new.iloc[train_index_new], y_new.iloc[test_index_new]

In [34]:
X_new_train.shape,y_new_train.shape

((227, 1, 4), (227,))

In [35]:
modelN = tf.keras.Sequential([
    tf.keras.layers.LSTM(100,activation='relu',input_shape =(1, 4)),
    tf.keras.layers.Dense(80,activation='relu'),
    tf.keras.layers.Dense(60,activation='relu'),
    tf.keras.layers.Dense(40,activation='relu'),
    tf.keras.layers.Dense(20,activation='relu'),
    tf.keras.layers.Dense(1)
])

C:\Users\Archit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [36]:
modelN.compile(optimizer='adam',loss='mae')

In [37]:
modelN.fit(X_new_train, y_new_train, epochs=100,validation_data=(X_new_test,y_new_test))

Epoch 1/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - loss: 1129.2435 - val_loss: 1172.1899
Epoch 2/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1132.2760 - val_loss: 1172.0830
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1126.5088 - val_loss: 1171.8628
Epoch 4/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1127.6660 - val_loss: 1171.3801
Epoch 5/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1127.5828 - val_loss: 1170.2906
Epoch 6/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1122.2479 - val_loss: 1167.8628
Epoch 7/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1124.4620 - val_loss: 1162.5964
Epoch 8/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1118.9703 - val_loss: 1151.6003
Epoch 9/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1106.9994 - val_loss: 1129.4602
Epoch 10/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1085.2496 - val_loss: 1086.4960
Epoch 11/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1038.6947 - val_loss: 1006.4219
Epoch 12/100
8/8 

In [38]:
modelN.evaluate(X_new_test,y_new_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 8.3836


8.38363265991211

In [39]:
y_pred_new = modelN.predict(X_new_test)
y_pred_new = np.argmax(y_pred_new, axis=1)
y_pred_new

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0], dtype=int64)

In [40]:
mae = mean_absolute_error(y_new_test, y_pred_new)
print(f"Mean Absolute Error: {mae}")

Mean Absolute Error: 1172.2475928


In [41]:
#Simple NN
foldNN = KFold(n_splits=10)

for train_index_NN, test_index_NN in foldNN.split(X_scaled, y):
    X_NN_train, X_NN_test = X_scaled[train_index_NN], X_scaled[test_index_NN]
    y_NN_train, y_NN_test = y.iloc[train_index_NN], y.iloc[test_index_NN]

In [42]:
X_NN_train.shape

(1133, 4)

In [43]:
modelNN = tf.keras.Sequential([
    
    tf.keras.layers.Dense(100,input_shape=(4,),activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(80,activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(60,activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(40,activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(20,activation='relu'),
    tf.keras.layers.Dense(1)
   
    
])

C:\Users\Archit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [44]:
modelNN.compile(optimizer='adam',loss='mae')

In [45]:
modelNN.fit(X_NN_train, y_NN_train, epochs=50,validation_data=(X_NN_test,y_NN_test))

Epoch 1/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 708.5707 - val_loss: 759.4943
Epoch 2/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 684.3484 - val_loss: 356.5229
Epoch 3/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 324.8463 - val_loss: 39.6763
Epoch 4/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 210.3798 - val_loss: 8.4768
Epoch 5/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 164.0563 - val_loss: 12.5090
Epoch 6/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 140.1776 - val_loss: 40.2549
Epoch 7/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 137.4307 - val_loss: 53.5115
Epoch 8/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 135.3979 - val_loss: 53.9522
Epoch 9/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 132.1043 - val_loss: 13.5331
Epoch 10/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 132.7327 - val_loss: 64.2031
Epoch 11/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 134.0595 - val_loss: 38.7734
Epoch 12/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0

In [46]:
modelNN.evaluate(X_NN_test, y_NN_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 36.0007 


36.1644401550293

In [47]:
y_pred_NN = modelNN.predict(X_NN_test)
y_pred_NN = np.argmax(y_pred_NN,axis=1)
y_pred_NN

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/stepWARNING:tensorflow:5 out of the last 9 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x000001F3B82F4400> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [48]:
mae = mean_absolute_error(y_NN_test, y_pred_NN)
print(f"Mean Absolute Error: {mae}")

Mean Absolute Error: 770.3359189453125


In [49]:
#Running ML models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [50]:
X_ML_train,X_ML_test,y_ML_train,y_ML_test = train_test_split(X,y,test_size=0.3,random_state=45)

In [51]:
cross_val_score(LinearRegression(),X_ML_train,y_ML_train)

array([-0.01913106,  0.22099155,  0.19525135,  0.2457819 ,  0.17626041])

In [52]:
cross_val_score(RandomForestRegressor(),X_ML_train,y_ML_train)

array([0.64602149, 0.53384068, 0.61511923, 0.78565885, 0.54831106])

In [53]:
cross_val_score(SVR(),X_ML_train,y_ML_train)

array([-0.00443972, -0.00780445, -0.014603  , -0.01500759, -0.02743258])

In [54]:
cross_val_score(DecisionTreeRegressor(),X_ML_train,y_ML_train)

array([0.57135691, 0.39458728, 0.32635494, 0.75292509, 0.05972505])

In [55]:
lr = LinearRegression()
lr.fit(X_ML_train,y_ML_train)
lr.score(X_ML_test,y_ML_test)

0.19922113418579102

In [56]:
rf = RandomForestRegressor()
rf.fit(X_ML_train,y_ML_train)
rf.score(X_ML_test,y_ML_test)

0.7369910380652189

In [57]:
svr = SVR()
svr.fit(X_ML_train,y_ML_train)
svr.score(X_ML_test,y_ML_test)

0.011322594519208584

In [58]:
dt = DecisionTreeRegressor()
dt.fit(X_ML_train,y_ML_train)
dt.score(X_ML_test,y_ML_test)

0.6114425147985112